In [1]:
!pip install transformers

In [2]:
import os
import time
import random
import numpy as np
import pandas as pd

from sklearn.metrics import roc_curve, auc

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from transformers import DistilBertForSequenceClassification
from transformers import AdamW
from transformers import DistilBertTokenizerFast

In [3]:
RANDOM_SEED = 2020
torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

DATA_PATH = "/content/"

## Dataset

In [4]:
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [5]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_df = pd.read_csv(os.path.join(DATA_PATH, "sat_train.tsv"), sep="\t")
valid_df = pd.read_csv(os.path.join(DATA_PATH, "sat_valid.tsv"), sep="\t")
test_df = pd.read_csv(os.path.join(DATA_PATH, "sat_test.tsv"), sep="\t")

train_encodings = tokenizer(train_df["context"].values.tolist(), truncation=True, padding=True)
valid_encodings = tokenizer(valid_df["context"].values.tolist(), truncation=True, padding=True)
test_encodings = tokenizer(test_df["context"].values.tolist(), truncation=True, padding=True)

train_dataset = CustomDataset(train_encodings, train_df["label"].values)
valid_dataset = CustomDataset(valid_encodings, valid_df["label"].values)
test_dataset = CustomDataset(test_encodings, test_df["label"].values)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

## Test function

In [ ]:
def bert_test(
    model: nn.Module,
    iterator: DataLoader
):

    with torch.no_grad():
        y_real = []
        y_pred = []
        model.eval()

        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            output = model(input_ids, attention_mask=attention_mask)[0]
            y_pred += [output]
            y_real += [batch["labels"]]
            
        y_real = torch.cat(y_real)
        y_pred = torch.cat(y_pred)[:,1]

    fpr, tpr, _ = roc_curve(y_real, y_pred)
    auroc = auc(fpr, tpr)

    return auroc

## Before fine-tuning

In [6]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
_ = model.to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [8]:
test_auroc = bert_test(model, test_loader)
test_auroc

0.38888888888888884

## Fine tuning

In [9]:
model.train()
optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(10):
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()
    test_auroc = bert_test(model, test_loader)
    print(f'| Test AUROC: {test_auroc:.5f}')    

| Test AUROC: 0.72222
| Test AUROC: 0.66667
| Test AUROC: 0.74747
| Test AUROC: 0.78788
| Test AUROC: 0.66667
| Test AUROC: 0.79798
| Test AUROC: 0.81313
| Test AUROC: 0.80303
| Test AUROC: 0.79798
| Test AUROC: 0.80303


In [10]:
test_auroc = bert_test(model, test_loader)
test_auroc

0.803030303030303